In [1]:
# Set the work directory to the imaginaire root.
import os, sys, time
import pathlib
root_dir = pathlib.Path().absolute().parents[2]
os.chdir(root_dir)
print(f"Root Directory Path: {root_dir}")

Root Directory Path: /home/cloud/wxz/neuralangelo


In [2]:
# Import Python libraries.
import numpy as np
import torch
import k3d
import json
from collections import OrderedDict
# Import imaginaire modules.
from projects.nerf.utils import camera, visualize
from third_party.colmap.scripts.python.read_write_model import read_model

In [3]:

# def gen_rot_view(elev, max_angle, distance, size, axis=[0, 1, 0]):

#     # bg_to_cam

#     cam_traj = get_rotating_cam(size, distance=distance, axis=axis, max_angle=max_angle)
#     cam_elev = get_object_to_camera_matrix(elev, axis, 0)[None]
#     cam_traj = cam_traj @ cam_elev
#     # # field2cam = create_field2cam(cam_traj, field2cam_fr.keys())

#     # camera_int = np.zeros((len(frameid_sub), 4))

#     # # focal length = img height * distance / obj height
#     # camera_int[:, :2] = opts["render_res"] * 2 * 0.8  # zoom out a bit
#     # camera_int[:, 2:] = opts["render_res"] / 2
#     return cam_traj

# def get_rotating_cam(num_cameras, axis=[0, 1, 0], distance=3, initial_angle=0, max_angle=360):
#     """Generate camera sequence rotating around a fixed object

#     Args:
#         num_cameras (int): Number of cameras in sequence
#         axis (ndarray): (3,) Axis of rotation
#         distance (float): Distance from camera to object
#         initial_angle (float): Initial rotation angle, degrees (default 0)
#         max_angle (float): Final rotation angle, degrees (default 360)
#     Returns:
#         extrinsics (ndarray): (num_cameras, 3, 4) Sequence of camera extrinsics
#     """
#     angles = np.linspace(initial_angle, max_angle, num_cameras)
#     extrinsics = np.zeros((num_cameras, 4, 4))
#     for i in range(num_cameras):
#         extrinsics[i] = get_object_to_camera_matrix(angles[i], axis, distance)
#     return extrinsics
# import cv2
# def get_object_to_camera_matrix(theta, axis, distance):
#     """Generate 3x4 object-to-camera matrix that rotates the object around
#     the given axis

#     Args:
#         theta (float): Angle of rotation in radians.
#         axis (ndarray): (3,) Axis of rotation
#         distance (float): Distance from camera to object
#     Returns:
#         extrinsics (ndarray): (3, 4) Object-to-camera matrix
#     """
#     theta = theta / 180 * np.pi
#     rt4x4 = np.eye(4)
#     axis = np.asarray(axis)
#     axis = axis / np.linalg.norm(axis)
#     # import ipdb; ipdb.set_trace()
#     R, _ = cv2.Rodrigues(theta * axis)
#     t = np.asarray([0, 0, distance])
#     rtmat = np.concatenate((R, t.reshape(3, 1)), axis=1)
#     rt4x4[:3, :4] = rtmat
#     return rt4x4

In [4]:
# Read the COLMAP data.
colmap_path = "/home/cloud/datasets/scan_head"
cameras, images, points_3D = read_model(path=f"{colmap_path}/dense/sparse", ext=".bin")
# Convert camera poses.
images = OrderedDict(sorted(images.items()))
qvecs = torch.from_numpy(np.stack([image.qvec for image in images.values()]))
tvecs = torch.from_numpy(np.stack([image.tvec for image in images.values()]))
Rs = camera.quaternion.q_to_R(qvecs)
poses = torch.cat([Rs, tvecs[..., None]], dim=-1)  # [N,3,4]
print(f"# images: {len(poses)}")
# Get the sparse 3D points and the colors.
xyzs = torch.from_numpy(np.stack([point.xyz for point in points_3D.values()]))
print("avg xyz", xyzs.mean(dim=0))
rgbs = np.stack([point.rgb for point in points_3D.values()])
rgbs = (rgbs[:, 0] * 2**16 + rgbs[:, 1] * 2**8 + rgbs[:, 2]).astype(np.uint32)
print(f"# points: {len(xyzs)}")

# images: 31
avg xyz tensor([-0.0218,  0.4226,  0.4587], dtype=torch.float64)
# points: 6573


In [5]:
# Visualize the bounding sphere.
json_fname = f"{colmap_path}/dense/transforms.json"
with open(json_fname) as file:
    meta = json.load(file)

center = meta["sphere_center"]
radius = meta["sphere_radius"]
# ------------------------------------------------------------------------------------
# These variables can be adjusted to make the bounding sphere fit the region of interest.
# The adjusted values can then be set in the config as data.readjust.center and data.readjust.scale
readjust_center = np.array([0, 0., -0.2])
readjust_scale = 0.6
# ------------------------------------------------------------------------------------
center += readjust_center
radius *= readjust_scale
# Make some points to hallucinate a bounding sphere.
sphere_points = np.random.randn(100000, 3)
sphere_points = sphere_points / np.linalg.norm(sphere_points, axis=-1, keepdims=True)
sphere_points = sphere_points * radius + center

# --------------------------------------------------------------
# ring_radius = 10
# ring_points = np.random.randn(10000, 3)
# ring_points[:, 1] = 0

# ring_points = ring_points / np.linalg.norm(ring_points, axis=-1, keepdims=True)
# ring_points = ring_points * ring_radius

# --------------------------------------------------------------
axis = [0, 1, 0] / np.sqrt(3)
# rot_poses = gen_rot_view(0, 360, 10, 60, axis)
import projects.neuralangelo.data
# rot_poses = projects.neuralangelo.data.get_rotating_cam(30, axis=axis, distance=10, initial_angle=0, max_angle=360)
# M(from meshlab to align object to XYZ axis)



def get_cam_pointing_p(cam_pos=[0.0, 0.0, 5.0], point=[0.0, 0.0, 0.0]):
    camera_position = torch.tensor([cam_pos])
    camera_direction = torch.tensor([point]) - camera_position  # point camera towards the origin
    camera_right = torch.cross(torch.tensor([[0.0, 1.0, 0.0]]), camera_direction)

    camera_direction = camera_direction / torch.norm(camera_direction, dim=1, keepdim=True)  # normalize direction vector
    camera_right = camera_right / torch.norm(camera_right, dim=1, keepdim=True)  # normalize right vector
    camera_up = torch.cross(camera_direction, camera_right)

    # Compute the camera pose
    R = torch.stack([camera_right[0], camera_up[0], camera_direction[0]], dim=1)
    # T = -torch.matmul(R, camera_position.transpose(0, 1)).squeeze()
    infer_pose = torch.eye(4, dtype=torch.float32)

    infer_pose[:3, :3] = R
    # import ipdb; ipdb.set_trace()
    infer_pose[:3, 3] = camera_position[0]
    # infer_pose = infer_pose[:3, :]
    return infer_pose

infer_pose = []

for x in range(1, 8):
    x = x * -0.5
    # infer_pose.append(get_cam_pointing_p(cam_pos=[x * 1., 0., 0.])[:3, :])
    # infer_intr.append(template_intr.clone())
    # infer_pose.append(get_cam_pointing_p(cam_pos=[0., x * 1., 0.])[:3, :])
    # infer_intr.append(template_intr.clone())
    p = get_cam_pointing_p(cam_pos=[0., 0., x], point=[0, 0, -1]) # 0.97, 1.75, 4.9
    infer_pose.append(p)
rot_poses = torch.stack(infer_pose, dim=0)

M = np.array([[27,-11.6,4.9,68.4], [12.5,22.7,-15,80], [2.2,15.7,25.5,-152.3],[0,0,0,30]])
M = torch.tensor(M / 30, dtype=torch.float64)

xyzs_homo = torch.cat([xyzs, torch.ones(xyzs.shape[0], 1)], dim=1)
xyzs_homo = (M @ xyzs_homo.T).T
xyzs_home = xyzs_homo - xyzs_homo.mean(axis=0)
xyzs_concat = torch.concat([xyzs, xyzs_homo[:, :3] / xyzs_homo[:, 3:4]])
# xyzs_concat = xyzs_homo[:, :3] / xyzs_homo[:, 3:4]
rgbs_concat = np.stack([rgbs, rgbs])
# rot_poses = np.matmul(rot_poses, M)
rot_poses = torch.tensor(rot_poses[:, :3, :4], dtype=torch.float64)

# template_data = torch.load("/home/cloud/wxz/neuralangelo/eval_data.pt")
# template_pose = template_data["pose"].to(M.device)
# rot_poses = template_pose

/tmp/ipykernel_3575601/2942947300.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rot_poses = torch.tensor(rot_poses[:, :3, :4], dtype=torch.float64)


In [7]:
# Visualize with K3D.
vis_scale = 0.5
plot = visualize.k3d_visualize_pose(poses,
                                    vis_depth=(0.5 * vis_scale),
                                    xyz_length=(0.1 * vis_scale),
                                    center_size=(0.05 * vis_scale),
                                    xyz_width=(0.02 * vis_scale))
# print(rot_poses)
# plot = visualize.k3d_visualize_pose(rot_poses,
#                                     vis_depth=(0.5 * vis_scale),
#                                     xyz_length=(0.1 * vis_scale),
#                                     center_size=(0.05 * vis_scale),
#                                     xyz_width=(0.02 * vis_scale))
plot += k3d.points(xyzs_concat, colors=rgbs_concat, point_size=(0.05 * vis_scale), shader="flat")
plot += k3d.points(sphere_points, color=0x4488ff, point_size=0.02, shader="flat")
# plot += k3d.points(ring_points, color=0xff2400, point_size=0.02, shader="flat")
plot.display()
plot.camera_fov = 30.0

Output()